In [ ]:
import pandas as pd
import numpy as np
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
import glob
import random
import sys
import networkx as nx
import matplotlib.pyplot as plt
# import folium
import osmnx as ox
import pandana as pdna
import libpysal as lps
import seaborn as sns
import mapclassify as mc
# import esda
from difflib import SequenceMatcher
from pathlib import Path
import statsmodels.api as sm
from scipy import stats
from statsmodels.graphics.api import abline_plot

import six
import uuid
import pickle
from sqlalchemy import create_engine

engine = create_engine('postgresql://postgres@localhost:5432/david')

from matplotlib import colors
from matplotlib.collections import LineCollection
from shapely.geometry import Point, Polygon
from utils import show_values
pd.set_option('display.max_columns', 500)

sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
# import pyspace

## Import FDGC's data

In [ ]:
#Set working directory
mydir = Path(os.getcwd())
data_folder = mydir / '../Data' #Set data folder
result_folder = mydir / '../Results' #Set data folder

In [ ]:
file = data_folder /'Raw data'/ "extractionGE_v3.csv" #Data source file containing the screening data
df = pd.read_csv(file,sep = ";",low_memory = False) #Load data

In [ ]:
len(df.numeroDossier.unique())

## Import administrative gdfs

In [ ]:
lake = gpd.read_file('/Users/david/Dropbox/PhD/GitHub/COVID19/Data/Mapping/lake.geojson')
cantons = gpd.read_file(
    '/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp')
communes = gpd.read_file(
    '/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.shp')
communes = communes[communes.KANTONSNUM == 25]
cantons = cantons.to_crs(2056)
communes = communes.to_crs(2056)

## Feature engineering

In [ ]:
df['dateInvitation'] = pd.to_datetime(df['dateInvitation'])
df['dateMammographie'] = pd.to_datetime(df['dateMammographie'])

df['year_invit'] = df.dateInvitation.dt.year
df['year_mammo'] = df.dateMammographie.dt.year

In [ ]:
df[df.year_invit.isnull()]

In [ ]:
df = df.assign(mammo = 1* (df.dateMammographie.isnull()==False))

In [ ]:
df.groupby('year_mammo').mammo.sum()

## Pre-geocoding address cleaning

In [ ]:
#Set all column names to lowercase
df.columns = map(str.lower, df.columns)

#eliminate noisy information from addresses
df.loc[df.adresse1.str.contains('C/O')==True, 'adresse1'] = '' 
df.loc[df.adresse1.str.contains('CASE POSTALE')==True, 'adresse1'] = ''
df.loc[df.adresse2.str.contains('CASE POSTALE')==True, 'adresse2'] = ''
df.loc[df.adresse2.isnull()==True, 'adresse2'] = ''

#Create new column containing country information
df['country'] = 'Suisse'
df.loc[df.npa.str.contains('F-|FR-')==True, 'country'] = 'France'
df.loc[df.localité.str.contains('FRANCE')==True, 'country'] = 'France'

# df['city'] = df['city'].astype(str)
# df.loc[df.city == 'Athenaz (Avusy)','city'] = 'Avusy'
# df.loc[df.city == 'Chambésy','city'] ='Pregny-Chambésy'
# df.loc[df.city.str.contains('Lancy'),'city'] = 'Lancy'
# df.loc[df.city == 'Collex','city'] = 'Collex-Bossy'
# df.loc[df.city == 'La Croix-de-Rozon','city'] = 'Bardonnex'
# df.loc[df.city == 'Perly','city'] = 'Perly-Certoux'
# df.loc[df.city.str.contains('Meyrin'),'city'] = 'Meyrin'
# df.loc[df.city.str.contains('Grand-Lancy'),'city'] = 'Grand-Lancy'
# df.loc[df.city.str.contains('Petit-Lancy'),'city'] = 'Petit-Lancy'
#Repare npa field
df['npa'] = df['npa'].str.replace('F-','').str.replace('FR -','').str.replace('F -','').str.replace('F','').str.replace('-','').str.strip().replace('',np.nan)
df['npa'] = df.npa.fillna(-1).astype(float).astype(int)
df['npa'] = df['npa'].astype(str).replace('-1',np.nan)

#Create new column : street
df['street'] = df['adresse1'] +' '+ df['adresse2']
df['street'] = df['street'].replace(',','', regex = True)

df = df.rename(columns = {'npa':'zipcode','localité':'city'})
df['canton'] = 'ge'
df.to_feather(data_folder/'Raw data'/'first_cleanup_giracs.feather')

_regbl_data_folder = Path("/Users/david/Dropbox/PhD/GitHub/Swiss-Geocoder/").resolve()
regbl_address = pd.read_feather(_regbl_data_folder / 'Data' /'Clean_data'/ 'regbl_address_2022.feather')
regbl_street = pd.read_feather(_regbl_data_folder / 'Data' /'Clean_data'/ 'regbl_street_2022.feather')

## Post-geocoding 

Geocoding using in-house procedure based on string-matching against a CH Federal database on all existing addresses in the country.

In [ ]:
### Geocoded dataset
df_geocoded = pd.read_excel(data_folder/'Processed data'/'giracs_geocoded_80.xlsx')
df_geocoded = df_geocoded.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_geocoded.loc[df_geocoded.city.str.contains('Meyrin', na=False),'city'] = 'Meyrin'
df_geocoded.loc[df_geocoded.city.str.contains('Lancy', na=False),'city'] = 'Lancy'
df_geocoded.loc[df_geocoded.city.str.contains('Genève', na=False),'city'] = 'Genève'

In [ ]:
#Transform variable to binary
df_geocoded.loc[df_geocoded.medecin == 'oui','medecin_code'] =1
df_geocoded.loc[df_geocoded.medecin == 'non','medecin_code'] =0
df_geocoded.loc[df_geocoded.medecin == 'je ne sais pas','medecin_code'] = np.nan

In [ ]:
#Transform variable to binary
df_geocoded.loc[df_geocoded.autremedecin == 'oui','autremedecin_code'] =1
df_geocoded.loc[df_geocoded.autremedecin == 'non','autremedecin_code'] =0
df_geocoded.loc[df_geocoded.autremedecin == 'je ne sais pas','autremedecin_code'] = np.nan

In [ ]:
#Transform variable to binary
df_geocoded.loc[df_geocoded.mammoanterieure == 'oui','mammoanterieure_code'] =1
df_geocoded.loc[df_geocoded.mammoanterieure == 'non','mammoanterieure_code'] =0
df_geocoded.loc[df_geocoded.mammoanterieure == 'je ne sais pas','mammoanterieure_code'] = np.nan

In [ ]:
#Transform variable to binary
df_geocoded.loc[df_geocoded.atf == 'oui','atf_code'] =1
df_geocoded.loc[df_geocoded.atf == 'non','atf_code'] =0
df_geocoded.loc[df_geocoded.atf == 'je ne sais pas','atf_code'] = np.nan

In [ ]:
#Transform variable to binary
rappel = 1* (df_geocoded.daterappel.isnull()==False)
df_geocoded = df_geocoded.assign(rappel = rappel)

In [ ]:
#Transform variable to binary
df_geocoded = df_geocoded.assign(mammo = 1* (df_geocoded.datemammographie.isnull()==False))

In [ ]:
#Create variables from date columns
df_geocoded['dateinvitation'] = pd.to_datetime(df_geocoded['dateinvitation'])
df_geocoded['daterappel'] = pd.to_datetime(df_geocoded['daterappel'])
df_geocoded['datemammographie'] = pd.to_datetime(df_geocoded['datemammographie'])

In [ ]:
#Create variables from date columns
df_geocoded['t_invit_rappel'] = df_geocoded['daterappel'] - df_geocoded['dateinvitation']
df_geocoded['t_invit_rappel']= df_geocoded['t_invit_rappel']/np.timedelta64(1,'D')

df_geocoded['t_invit_mammo'] = df_geocoded['datemammographie'] - df_geocoded['dateinvitation']
df_geocoded['t_invit_mammo']= df_geocoded['t_invit_mammo']/np.timedelta64(1,'D')

In [ ]:
#Create variables from date columns
df_geocoded['year_invit'] = df_geocoded.dateinvitation.dt.year
df_geocoded['month_invit'] = df_geocoded.dateinvitation.dt.month
df_geocoded['day_invit'] = df_geocoded.dateinvitation.dt.day
df_geocoded['weekday_invit'] = df_geocoded.dateinvitation.dt.day_name()

df_geocoded['year_rappel'] = df_geocoded.daterappel.dt.year
df_geocoded['month_rappel'] = df_geocoded.daterappel.dt.month
df_geocoded['day_rappel'] = df_geocoded.daterappel.dt.day
df_geocoded['weekday_rappel'] = df_geocoded.daterappel.dt.day_name()

df_geocoded['year_mammo'] = df_geocoded.datemammographie.dt.year
df_geocoded['month_mammo'] = df_geocoded.datemammographie.dt.month
df_geocoded['day_mammo'] = df_geocoded.datemammographie.dt.day
df_geocoded['weekday_mammo'] = df_geocoded.datemammographie.dt.day_name()

In [ ]:
#Return a sequence that corresponds to the actual numeroinvitation ...without the weird things we find in the original column
df_geocoded['dt_invit'] = pd.to_datetime([f'{y}-{m}-{d}' for y, m, d in zip(df_geocoded.year_invit, df_geocoded.month_invit, df_geocoded.day_invit)])
df_geocoded = df_geocoded.sort_values(['numerodossier','dt_invit'])
df_geocoded['numeroinvitation_seq'] = df_geocoded.groupby('numerodossier').cumcount()+1

In [ ]:
n_invit = df_geocoded.groupby('numerodossier').numeroinvitation_seq.nunique()
df_geocoded = df_geocoded.join(n_invit, on='numerodossier', rsuffix='_n')

#### Participation changes between any two invitation (to the same woman)

In [ ]:
df_geocoded['mammo_last_invite'] = df_geocoded[['numerodossier','mammo']].groupby('numerodossier').diff()['mammo']

In [ ]:
participation_chg = pd.DataFrame(df_geocoded[['numerodossier','mammo_last_invite']].groupby(['mammo_last_invite']).numerodossier.count()).reset_index()
participation_chg.columns = ['Participation change','n']
participation_chg['%'] = participation_chg['n'].mul(100)/participation_chg['n'].sum(axis = 0)
participation_chg['%'] = participation_chg['%'].round(2)

In [ ]:
participation_change = df_geocoded[['numerodossier','mammo_last_invite']].dropna().groupby(['numerodossier']).mammo_last_invite.nunique().reset_index()
participation_change.columns = ['numerodossier','participation_change']
participation_change['participation_change'] = participation_change['participation_change']-1
df_geocoded = df_geocoded.merge(participation_change, on='numerodossier',how = 'left')

#### Time intervals between any two invitation (to the same woman)

In [ ]:
%%time
df_geocoded['diff_years'] = (df_geocoded
                            .sort_values(['numerodossier', 'dt_invit'])
                            .groupby('numerodossier')['dt_invit']
                            .diff()
                            .dt.days / 365.25)

In [ ]:
df_geocoded.shape

### Discard 2021 invitations

In [ ]:
df_geocoded.groupby('year_invit').numerodossier.count()

In [ ]:
df_geocoded = df_geocoded[df_geocoded.year_invit != 2021]
print('Number of invitations :', df_geocoded.shape[0])
print('Number of women :', df_geocoded.numerodossier.nunique())

### Discard duplicates

In [ ]:
clean_dupli =df_geocoded[(df_geocoded.duplicated(subset=['numerodossier','numeroinvitation'],keep=False))].sort_values(['numerodossier','month_invit','day_invit','mammo','groupeage']).drop_duplicates(subset = ['numerodossier','numeroinvitation'],keep = 'first')

df_geocoded['_dummy'] = df_geocoded['numerodossier'].astype(str) + df_geocoded['numeroinvitation'].astype(str)
clean_dupli['_dummy'] = clean_dupli['numerodossier'].astype(str) + clean_dupli['numeroinvitation'].astype(str)

df_geocoded = df_geocoded[(df_geocoded._dummy.isin(clean_dupli._dummy))==False]

df_geocoded = pd.concat([df_geocoded,clean_dupli])

print('Number of invitations :', df_geocoded.shape[0])
print('Number of women :', df_geocoded.numerodossier.nunique())

In [ ]:
579033/589879

In [ ]:
df_geocoded[df_geocoded.groupeage == '70-74'].groupby('year_invit').numerodossier.count()

### Discard out of age range invitations

In [ ]:
df_geocoded.groupeage = df_geocoded.groupeage.astype('category')
df_geocoded['groupeage_cat'] = df_geocoded.groupeage.cat.codes

In [ ]:
#Get age categories that are out of the age range guidelines for breast cancer screening
age_cats = df_geocoded.groupby('groupeage').size()
age_cats1000 = age_cats[age_cats > 1000]
age_cats =  pd.DataFrame(age_cats).reset_index()
age_cats.columns = ["Age group","Invitation count (n)"]

In [ ]:
df_geocoded = df_geocoded[(df_geocoded.groupeage.isin(['50-54','55-59','60-64','65-69','70-74']))]

In [ ]:
print('Number of invitations :', df_geocoded.shape[0])
print('Number of women :', df_geocoded.numerodossier.nunique())

In [ ]:
# Exclude 70-74 invited before 2013 since it wasn't rolled out yet
df_geocoded = df_geocoded[~((df_geocoded['year_invit'] < 2013) & (df_geocoded['groupeage'] == '70-74'))]

In [ ]:
print('Number of invitations :', df_geocoded.shape[0])
print('Number of women :', df_geocoded.numerodossier.nunique())

In [ ]:
#Create geodataframe
gdf_geocoded = gpd.GeoDataFrame(df_geocoded,crs = 2056,geometry = gpd.points_from_xy(df_geocoded['E'],df_geocoded['N']))
# Create geodataframe of unique addresses
gdf_geom_uniq = gdf_geocoded.drop_duplicates(subset = ['address_id','full_address'])
# Create geodataframe of all rows having a valid geocoding
gdf_geocoded_ok = gdf_geocoded[gdf_geocoded.comment.str.contains('Ok')]

In [ ]:
#Print the number of different geometries after small alteration to addresses
gdf_geocoded.groupby('comment').size()

In [ ]:
gdf_geocoded[~gdf_geocoded.comment.str.contains('Ok')].shape[0]

In [ ]:
gdf_geocoded[gdf_geocoded.comment.str.contains('Ok')].shape[0]

In [ ]:
print('Number of invitations :', gdf_geocoded_ok.shape[0])
print('Number of women :', gdf_geocoded_ok.numerodossier.nunique())

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(15, 6), sharey=False, sharex=True)
fig.suptitle('Geocoding Results', size = 18)
geocoding_res = pd.DataFrame(df_geocoded.comment.value_counts()).reset_index()
geocoding_res['count'] = geocoding_res['count'].astype(int)
chart = sns.barplot(x="comment", y="count", data=geocoding_res, ax = axes[0])
chart.set_xticklabels(chart.get_xticklabels(),size = 12, rotation=45, horizontalalignment='right')
chart.set_xlabel('',size = 14)
chart.set_ylabel('Count',size = 14)
chart = sns.barplot(x="comment", y="count", data=geocoding_res, ax = axes[1])
chart.set_yscale("log")
chart.set_xticklabels(chart.get_xticklabels(),size = 12, rotation=45, horizontalalignment='right')
chart.set_xlabel('Geocoding categories',size = 14)
chart.set_ylabel('Count (log scale)',size = 14)
show_values(chart, digits = 0)

geocoding_res = pd.DataFrame(df_geocoded.comment.value_counts(normalize = True).mul(100)).reset_index()
chart = sns.barplot(x="comment", y="proportion", data=geocoding_res, ax = axes[2])
chart.set_xticklabels(chart.get_xticklabels(),size = 12, rotation=45, horizontalalignment='right')
chart.set_xlabel('',size = 14)
chart.set_ylabel('Percentage',size = 14)
show_values(chart, digits = 1)

## Accessibility analyses

### Screening centers

In [ ]:
# Collect addresses from the centers
dic_centre = {'HUG - Maternité':[46.191174, 6.146240],
              'ImageRive':[46.202599, 6.150099],
              'CIO':[46.185712, 6.105039],
              'Clinique des Grangettes':[46.199840, 6.181743],
              'Clinique Générale Beaulieu':[46.188670, 6.161817],
              'MédImage SA':[46.196916, 6.154018],
              "Centre d'Imagerie Rive Gauche CIRG":[46.240752, 6.199488],
              'Hôpital de la Tour':[46.228690, 6.066374],
              "Institut d'Imagerie Médicale SA":[46.194231, 6.144169],
              "Centre d'Imagerie de la Servette":[46.212972, 6.125423],
              'Affidea CDRC':[46.186589, 6.143063],
              "Centre d'imagerie Rive Droite SA":[46.208147, 6.143412],
              'Imagerie Médicale Battikha':[46.210023, 6.143956],
              'Cabinet de radiologie Drs Berger & Teicher':[46.213017, 6.131947],
              'Imagerive LAC':[46.216705, 6.148251],
              'Cabinet de radiologie Dr Guinand-Mkinsi':[46.195798, 6.154703],
              'Clinique de la Colline':[46.191812, 6.149944],
              'Cabinet de radiologie Dr Arroja':[46.199595, 6.155487],
              'Centre Imagerie Versoix':[46.28096883497587, 6.165184209801708],
             "CIMG centre Imagerie médicale Genève":[46.216800901902246, 6.148267060384218]}

In [ ]:
df_centre =  pd.DataFrame.from_dict(dic_centre).T.reset_index()
df_centre.columns = ['center_name','lat','lon']

In [ ]:
df_centre.loc[df_centre['center_name'].isin(["Cabinet de radiologie Dr Guinand-Mkinsi",
"Cabinet de radiologie Drs Berger & Teicher",
"Affidea CDRC",
"Clinique des Grangettes",
"Clinique Générale Beaulieu",
"Hôpital de la Tour",
"HUG - Maternité",
"Imagerie Médicale Battikha",
"ImageRive",
"Institut d'Imagerie Médicale SA",
"Rive Droite SA - Centre d'imagerie SA",
"Centre d'imagerie Rive Droite SA",
"Clinique de la Colline",                                    
"Cabinet de radiologie Dr Arroja"]),'year_start'] = 1999
df_centre.loc[df_centre['center_name'].isin(['MédImage SA']),'year_start'] = 2010
df_centre.loc[df_centre['center_name'].isin(['CIO']),'year_start'] = 2012
df_centre.loc[df_centre['center_name'].isin(["Centre d'Imagerie Rive Gauche CIRG"]),'year_start'] = 2013
df_centre.loc[df_centre['center_name'].isin(["Centre d'Imagerie de la Servette"]),'year_start'] = 2016
df_centre.loc[df_centre['center_name'].isin(["CIMG centre Imagerie médicale Genève"]),'year_start'] = 2016
df_centre.loc[df_centre['center_name'].isin(["Imagerive LAC"]),'year_start'] = 2019
df_centre.loc[df_centre['center_name'].isin(["Centre Imagerie Versoix"]),'year_start'] = 2020
df_centre.loc[df_centre['center_name'].isin(['Cabinet de radiologie Drs Berger & Teicher']),'year_end'] = 2016
df_centre.loc[df_centre['center_name'].isin(['Cabinet de radiologie Dr Guinand-Mkinsi']),'year_end'] = 2013

In [ ]:
# gdf_centre = pyspace.make_gdf(df_centre,4326,'lat','lon')
gdf_centre = gpd.GeoDataFrame(df_centre, crs= 4326, geometry = gpd.points_from_xy(df_centre['lon'],df_centre['lat']))
gdf_centre.to_file(data_folder/'Raw data'/'BC_ScreeningCenters.geojson',driver = 'GeoJSON')

In [ ]:
gdf_centre.index = gdf_centre.index.map(int)
gdf_centre['lon'] = gdf_centre.geometry.x
gdf_centre['lat'] = gdf_centre.geometry.y
gdf_centre['id'] = gdf_centre.index+1000000
gdf_centre['quantity'] = 1.0
gdf_centre['category'] = 'centre'
gdf_centre = gdf_centre.reset_index(drop=True)

In [ ]:
gdf_centre = gpd.GeoDataFrame(gdf_centre, crs= 4326, geometry = gpd.points_from_xy(gdf_centre['lon'],gdf_centre['lat']))

### Import road network

In [ ]:
# # If network not previously saved
# G = ox.graph_from_place('Canton de Genève, Switzerland', network_type='all', simplify= True, clean_periphery = True)
# # Otherwise
# with open(data_folder/'G.gpickle', 'rb') as f:
#     G = pickle.load(f)


misc_data_folder = Path(data_folder/'Misc data')
gpickle_path = misc_data_folder / 'G.gpickle'

# Check if the network has been previously saved
if not gpickle_path.exists():
    # If not, download and save the network
    G = ox.graph_from_place('Canton de Genève, Switzerland', network_type='all', simplify=True)
    with open(gpickle_path, 'wb') as f:
        pickle.dump(G, f)
    print("Network downloaded and saved.")
else:
    # Otherwise, load the network from the saved file
    with open(gpickle_path, 'rb') as f:
        G = pickle.load(f)
    print("Network loaded from saved file.")

In [ ]:
# Project networks
Gp_4326 = ox.project_graph(G, to_crs = 4326)
Gp_2056 = ox.project_graph(G, to_crs = 2056)

### Calculation of access measures

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
gdf_edges = gdf_edges.reset_index()

In [ ]:
ax = gdf_nodes.plot(markersize = 1)
gdf_centre.plot(markersize = 10, ax = ax)

In [ ]:
# Get nearest 2 POIs (screening center) at max 5km
net=pdna.Network(gdf_nodes["x"], gdf_nodes["y"], gdf_edges["u"], gdf_edges["v"],
                 gdf_edges[["length"]])
net.precompute(5000)
net.set_pois(category='centre', maxdist = 5000, maxitems=len(gdf_centre), x_col=gdf_centre['lon'], y_col=gdf_centre['lat'])
nearest_2 = net.nearest_pois(5000, "centre", num_pois=2)

In [ ]:
# Get POI density (screening centers) within 5km

x, y = gdf_centre.lon, gdf_centre.lat

gdf_centre["node_ids"] = net.get_node_ids(x, y)
net.set(gdf_centre["node_ids"], variable=gdf_centre['quantity'], name="centre")
density5000 = net.aggregate(5000, type="sum", decay="linear", name="centre")

In [ ]:
net.plot(nearest_2[1],
         fig_kwargs={'figsize': [10, 10]},
         plot_kwargs={'cmap': 'RdYlGn_r', 's': 8, 'edgecolor': 'none'})

In [ ]:
nn_xy = ox.distance.nearest_nodes(Gp_2056, gdf_geocoded_ok.geometry.x, gdf_geocoded_ok.geometry.y)

In [ ]:
gdf_geocoded_ok['nearest_node'] = nn_xy
gdf_geocoded_ok['center_nearest'] = gdf_geocoded_ok['nearest_node'].map(nearest_2[1].to_dict())

In [ ]:
gdf_geocoded_ok['center_density'] = gdf_geocoded_ok['nearest_node'].map(density5000.to_dict())

In [ ]:
gdf_geocoded_ok_waccess = gdf_geocoded_ok.copy()

In [ ]:
gdf_geocoded_ok_waccess = gpd.sjoin(gdf_geocoded_ok_waccess, cantons[cantons.NAME == 'Genève'][['geometry']], predicate = 'intersects')
gdf_geocoded_ok_waccess = gdf_geocoded_ok_waccess.drop('index_right', axis = 1)

In [ ]:
print('Number of invitations :', gdf_geocoded_ok_waccess.shape[0])
print('Number of women :', gdf_geocoded_ok_waccess.numerodossier.nunique())

In [ ]:
gdf_geocoded_ok_waccess[gdf_geocoded_ok_waccess.year_invit == 2019].plot('center_nearest', markersize = 1)

## SES Deprivation index

In [ ]:
from sqlalchemy import text

In [ ]:
query = """select * from data_raw.microgis_geo_gva where geom is not null ;"""

# s_settings_df = pd.DataFrame(engine_cloud.connect().execute(text(query)))

with engine.connect() as conn: 
                                                       
    microgis_geo = gpd.GeoDataFrame.from_postgis(text(query), con = conn, geom_col = 'geom')
# microgis_data = microgis_data.dropna()
microgis_geo.crs = 'epsg:2056'



In [ ]:
query = """select nbid,"LOCALITY" locality,ciqmd,ptot,pm,pf,p0004,p0509,p1014,p1519,p2024,p2529,p3034,p3539,p4044,p4549,p5054,p5559,p6064,p6569,p7074,p7579,p8084,p8589,p90m,rpmssin,rpmsmar,rpmswid,rpmsdiv,rad3prim, rad3sec,rad3tert,rprprot,rprcath,rprochr,rprjew,rprmusl,rproth,rprnorel,rpnch,
rpnoce,
rpncam,
rpncas,
rpnceu,rpneeu,rpneceu,rpnfe,rpnme,rpnnaf,rpnnam,rpnneu,rpnnweu,rpnsam,rpnsas,rpnseas,rpnseeu,rpnsweu,rpnsubaf,rpnweu
,rphhpriv,rhhcoll,rhhp1p,rhhp2p,rhhp3p,rhhp4p,rhhp5p,rhhp6mp,rpfnone,rpfobl,rpfgen,rpfprof,rpfmat,rpfprsf,rpfprss ,rpfbac,rpfmas,rpfphd,rad,radf,radune,rado,radunef,radslib,dmdrent,b.geom geometry from data_raw.microgis_data_gva a, data_raw.microgis_geo_gva b where a.nbid = b."NBID" and b.geom is not null ;"""


with engine.connect() as conn: 
                                                       
    microgis_data = gpd.GeoDataFrame.from_postgis(text(query), con = conn, geom_col = 'geometry')
microgis_data.crs = 'epsg:2056'

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import text

## Deprivation index

query = """select nbid,"LOCALITY" locality,ciqmd,ptot,pm,pf,p0004,p0509,p1014,p1519,p2024,p2529,p3034,p3539,p4044,p4549,p5054,p5559,p6064,p6569,p7074,p7579,p8084,p8589,p90m,rpmssin,rpmsmar,rpmswid,rpmsdiv,rad3prim, rad3sec,rad3tert,rprprot,rprcath,rprochr,rprjew,rprmusl,rproth,rprnorel,rpnch,
rpnoce,
rpncam,
rpncas,
rpnceu,rpneeu,rpneceu,rpnfe,rpnme,rpnnaf,rpnnam,rpnneu,rpnnweu,rpnsam,rpnsas,rpnseas,rpnseeu,rpnsweu,rpnsubaf,rpnweu
,rphhpriv,rhhcoll,rhhp1p,rhhp2p,rhhp3p,rhhp4p,rhhp5p,rhhp6mp,rpfnone,rpfobl,rpfgen,rpfprof,rpfmat,rpfprsf,rpfprss ,rpfbac,rpfmas,rpfphd,rad,radf,radune,rado,radunef,radslib,dmdrent,b.geom geometry from data_raw.microgis_data_gva a, data_raw.microgis_geo_gva b where a.nbid = b."NBID" and b.geom is not null ;"""

# s_settings_df = pd.DataFrame(engine_cloud.connect().execute(text(query)))

with engine.connect() as conn: 
                                                       
    microgis_data = gpd.GeoDataFrame.from_postgis(text(query), con = conn, geom_col = 'geometry')
# microgis_data = microgis_data.dropna()
microgis_data.crs = 'epsg:2056'



microgis_data['tertiary_education'] = microgis_data[['rpfbac','rpfmas','rpfphd']].sum(axis = 1)
microgis_data['rpforeign'] = 100 - microgis_data['rpnch']
microgis_data['wo_tertiary_education'] = 100-microgis_data['tertiary_education']
microgis_data['inv_ciqmd'] = -microgis_data['ciqmd']
microgis_data['inv_dmdrent'] = -microgis_data['dmdrent']
microgis_data['rad3primsec'] = 100 - microgis_data['rad3tert']
microgis_data['rp65+'] = (microgis_data.p6569+microgis_data.p7074+microgis_data.p7579+microgis_data.p8084+microgis_data.p8589+microgis_data.p90m)/microgis_data.ptot
microgis_data['rpdens'] = (microgis_data.ptot / microgis_data.geometry.area)*1000000
microgis_data['rphh3p-'] = microgis_data[['rhhp1p','rhhp2p','rhhp3p']].sum(axis = 1)
microgis_data['rphh3p+'] = 100 - microgis_data[['rhhp1p','rhhp2p','rhhp3p']].sum(axis = 1)
#Filter out any neighborhood having ptot = 0
microgis_data = microgis_data[microgis_data.ptot>0]

#Standardize selected variables
x = microgis_data[['rpforeign','inv_ciqmd','radune','wo_tertiary_education','inv_dmdrent','rad3primsec']].values
x = StandardScaler().fit_transform(x)


#PCA with 3 components
pca_depriv = PCA(n_components=3)
principalComponents_depriv = pca_depriv.fit_transform(x)

principal_depriv_Df = pd.DataFrame(data = principalComponents_depriv
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

print('Explained variation per principal component: {}'.format(pca_depriv.explained_variance_ratio_))


microgis_data['deprivation_pca'] = -principalComponents_depriv.T[0]

# Fit your data on the scaler object
scaler = preprocessing.StandardScaler()
microgis_data[['deprivation_pca']] = scaler.fit_transform(microgis_data[['deprivation_pca']])
microgis_data['deprivation_pca'] = -principalComponents_depriv.T[0]
microgis_data['deprivation_pca_q5'] = pd.qcut(microgis_data['deprivation_pca'],5, labels = False)
microgis_data.to_parquet(data_folder/'Processed data'/'microgis_data_depriv.parquet',index = False)

pd.DataFrame(pca_depriv.components_,columns=microgis_data[['rpforeign','inv_ciqmd','radune','wo_tertiary_education','inv_dmdrent','rad3primsec']].columns,index = ['PC-1','PC-2','PC-3'])


In [ ]:
#Make spatial join to assign area-based variables to the GIRACS individuals
gdf_geom_ok_waccess = gpd.sjoin(gdf_geocoded_ok_waccess, microgis_data[['nbid','geometry']], how = 'left', predicate = 'intersects').drop('index_right',axis = 1)

In [ ]:
gdf_geom_ok_waccess.loc[(gdf_geom_ok_waccess.year_invit> 2002) &(gdf_geom_ok_waccess['nbid'].isna())].shape[0]/4823.18

In [ ]:
def min_dist_calc(points, polygons):
    """
    Calculates the minimum distance from each point to the closest polygon when they don't intersect.
    Updates the 'nbid' in the points GeoDataFrame with the id of the closest polygon.
    """
    # Ensure 'nbid' is a float for nan checks
    points['nbid'] = points['nbid'].astype(float)

    # Iterate only over points without an 'nbid'
    for i, point in points.loc[points['nbid'].isna()].iterrows():
        min_distance = np.inf
        closest_nbid = None
        closest_locality = None
        
        # Check distance for each polygon
        for index, polygon in polygons.iterrows():
            geom = polygon.geometry
            if geom.geom_type == 'Polygon':
                dist = geom.exterior.distance(point.geometry)
            else:  # MultiPolygon case
                dist = min(poly.exterior.distance(point.geometry) for poly in geom.geoms)
            
            if dist < min_distance:
                min_distance, closest_nbid, closest_locality = dist, polygon.nbid, polygon.locality
        
        # Update points GeoDataFrame with the closest 'nbid' and print/log results
        points.at[i, 'nbid'] = closest_nbid
        # Consider replacing print with logging for production code
        print(closest_nbid, closest_locality)

min_dist_calc(gdf_geom_ok_waccess, microgis_data)

In [ ]:
columns_to_merge = [
    'nbid', 'deprivation_pca', 'rpdens', 'deprivation_pca_q5','ptot','ciqmd', 'pf',
    'p0004', 'p0509', 'p1014', 'p1519', 'p2024', 'p2529', 'p3034', 'p3539', 
    'p4044', 'p4549', 'p5054', 'p5559', 'p6064', 'p6569', 'p7074', 'p7579', 
    'p8084', 'p8589', 'p90m', 'rpnoce', 'rpncam', 'rpncas', 'rpnceu', 
    'rpneeu', 'rpneceu', 'rpnfe', 'rpnme', 'rpnnaf', 'rpnnam', 'rpnneu', 
    'rpnnweu', 'rpnsam', 'rpnsas', 'rpnseas', 'rpnseeu', 'rpnsweu', 
    'rpnsubaf', 'rpnweu', 'tertiary_education', 'rpforeign', 
    'wo_tertiary_education', 'inv_ciqmd', 'inv_dmdrent','dmdrent', 'rad3primsec', 
    'rp65+', 'rphh3p-', 'rphh3p+','rad3sec', 'rad3tert', 'rprprot', 'rprcath', 'rprochr', 'rprjew', 'rprmusl', 'rproth', 'rprnorel', 'rpnch', 'rphhpriv', 'rpfnone', 'rpfobl', 'rpfgen', 'rpfprof', 'rpfmat', 'rpfprsf', 'rpfprss', 'rpfbac', 'rpfmas', 'rpfphd', 'rad', 'radf', 'radune', 'radslib'

]

gdf_geocoded_ok_waccess_wmicrogis = pd.merge(
    gdf_geom_ok_waccess, 
    microgis_data[columns_to_merge], 
    on='nbid',
    how='left'
)

In [ ]:
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar
dict_cl = {np.nan:'#bababa',0:'#0571b0',
1:'#92c5de',
2:'#f7f7f7',
3:'#f4a582',
4:'#ca0020'}
hmap = colors.ListedColormap([dict_cl[i] for i in microgis_data['deprivation_pca_q5'].sort_values().unique()])


dict_cl = {np.nan:'#bababa',4:'#d7191c',
3:'#fdae61',
2:'#ffffbf',
1:'#abd9e9',
0:'#2c7bb6'}
microgis_data['depriv_color'] = microgis_data['deprivation_pca_q5'].map(dict_cl)

In [ ]:
microgis_data['depriv_color'].isna().sum()

In [ ]:
ax = microgis_data.plot('deprivation_pca_q5',alpha = 0.8,figsize = (10,10),cmap=hmap , linewidth = 0.2, edgecolor = 'k')
ax.set_title('Spatial distribution of the deprivation index in quintiles in the canton of Geneva, Switzerland.')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatter,crs = 'EPSG:2056')
# add scale bar
scalebar = ScaleBar(1, units="m", location="lower right")
ax.add_artist(scalebar)
ax.set_axis_off()
x, y, arrow_length = 0.9, 0.15, 0.06
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)
plt.savefig(result_folder/'Deprivation_cantongva.png',dpi = 800, bbox_inches='tight')

In [ ]:
ax = microgis_data.plot(alpha = 0.8,figsize = (10,10),color = microgis_data.depriv_color , linewidth = 0.2, edgecolor = 'k')
ax.set_title('Spatial distribution of the deprivation index in quintiles in the canton of Geneva, Switzerland.')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron,crs = 'EPSG:2056')
# add scale bar
scalebar = ScaleBar(1, units="m", location="lower right")
ax.add_artist(scalebar)
ax.set_axis_off()
x, y, arrow_length = 0.9, 0.15, 0.06
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)
plt.savefig(result_folder/'Deprivation_cantongva.png',dpi = 800)

In [ ]:
microgis_gva = microgis_data[microgis_data.locality == 'Genève']

In [ ]:
ax = microgis_gva.plot(alpha = 0.8,figsize = (10,10),color = microgis_gva.depriv_color, linewidth = 0.2, edgecolor = 'k')
ax.set_title('Spatial distribution of the deprivation index in quintiles in Geneva, Switzerland.')
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron,crs = 'EPSG:2056')
# add scale bar
scalebar = ScaleBar(1, units="m", location="lower right")
ax.add_artist(scalebar)
ax.set_axis_off()
x, y, arrow_length = 0.9, 0.15, 0.06
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)
plt.savefig(result_folder/'Deprivation_gva.png',dpi = 800)

In [ ]:
microgisSESindexR = pd.read_feather('../Data/Processed data/microgis_GE_indexSES.feather')

In [ ]:
microgis_data = pd.merge(microgis_data,microgisSESindexR[['nbid','index_socio_class']], on = 'nbid')
microgis_data['index_socio_class'] = microgis_data['index_socio_class'].astype(int)
microgis_data['depriv_color_pkg_Lalloue'] = microgis_data['index_socio_class'].map(dict_cl)

In [ ]:
microgis_data.index_socio_class.value_counts()

In [ ]:
# ax = microgis_data.plot(alpha = 0.8,figsize = (10,10),color = microgis_data.depriv_color_pkg_Lalloue , linewidth = 0.2, edgecolor = 'k')
# ax.set_title('Spatial distribution of the deprivation index in quintiles in the canton of Geneva, Switzerland.')
# ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron,crs = 'EPSG:2056')
# # add scale bar
# scalebar = ScaleBar(1, units="m", location="lower right")
# ax.add_artist(scalebar)
# ax.set_axis_off()
# x, y, arrow_length = 0.9, 0.15, 0.06
# ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
#             arrowprops=dict(facecolor='black', width=5, headwidth=15),
#             ha='center', va='center', fontsize=20,
#             xycoords=ax.transAxes)
# plt.savefig(result_folder/'Deprivation_cantongva.png',dpi = 800)

## Add median income at the sous-secteur scale

In [ ]:
ss_income = pd.read_excel('../../GeoSalt/Data/OCTSTAT_historique_revenus_2005_2016.xlsx',sheet_name = "Sheet1")

In [ ]:
query = """select * from geounits.geo_girec;"""
geo_girec = gpd.GeoDataFrame.from_postgis(query,con = engine,geom_col = 'geom')

In [ ]:
geo_girec['NUMERO'] = geo_girec['NUMERO'].astype(int)

ss_income = pd.merge(ss_income,geo_girec[['NUMERO','geom']],left_on = 'code_ss',right_on = 'NUMERO',how = 'left').drop('NUMERO',axis = 1)

ss_income = gpd.GeoDataFrame(ss_income,geometry = ss_income['geom'])
ss_income.crs = 'epsg:2056'

df_final = gpd.sjoin(gdf_geom_ok_waccess,ss_income, how = 'left',predicate = 'intersects').drop('index_right',axis = 1)

In [ ]:
df_final.loc[df_final.year_invit < 2006,'income_ss'] = df_final[2005]
df_final.loc[df_final.year_invit == 2006,'income_ss'] = df_final[2006]
df_final.loc[df_final.year_invit == 2007,'income_ss'] = df_final[2007]
df_final.loc[df_final.year_invit == 2008,'income_ss'] = df_final[2008]
df_final.loc[df_final.year_invit == 2009,'income_ss'] = df_final[2009]
df_final.loc[df_final.year_invit == 2010,'income_ss'] = df_final[2010]
df_final.loc[df_final.year_invit == 2011,'income_ss'] = df_final[2011]
df_final.loc[df_final.year_invit == 2012,'income_ss'] = df_final[2012]
df_final.loc[df_final.year_invit == 2013,'income_ss'] = df_final[2013]
df_final.loc[df_final.year_invit == 2014,'income_ss'] = df_final[2014]
df_final.loc[df_final.year_invit == 2015,'income_ss'] = df_final[2015]
df_final.loc[df_final.year_invit >= 2016,'income_ss'] = df_final[2016]

In [ ]:
# df_final = df_final[~df_final.income_ss.isnull()]
participants_after = df_final.shape[0]
print(f'After removing respondents who were missing income there were left {participants_after} participants.')
print('Number of people in the dataset: ',len(df_final.numerodossier.unique()))

## Final checks

In [ ]:
df_giracs = gdf_geocoded_ok_waccess_wmicrogis.copy()

In [ ]:
df_giracs['longitude'] = df_giracs.to_crs(epsg=4326).geometry.x
df_giracs['latitude'] = df_giracs.to_crs(epsg=4326).geometry.y

In [ ]:
df_giracs['age_cat'] = pd.factorize(df_giracs['groupeage'], sort=True)[0] + 1 

In [ ]:
df_giracs = pd.concat([df_giracs,pd.get_dummies(df_giracs.groupeage)],axis = 1)

In [ ]:
## Do we have any remaining NaNs?
# df_giracs.isna().sum().sort_values().tail(40)

In [ ]:
print('Number of invitations :', gdf_geocoded_ok_waccess.shape[0])
print('Number of women :', gdf_geocoded_ok_waccess.numerodossier.nunique())

In [ ]:
df_giracs.to_feather('../Data/Processed data/20250916_GIRACS_all.feather',index = False)